# Setup of SageMaker Endpoint for DescriptiveWorld computer vision model

Authored by Blair Jones, 2021.11.01

Model built using yolov5

Code inspired by AWS-provided example at: https://aws.amazon.com/blogs/machine-learning/speed-up-yolov4-inference-to-twice-as-fast-on-amazon-sagemaker/



### Instructions

Before running the notebook, install yolov5 in the Sagemaker root folder using the yolov5 instructions at:  https://github.com/ultralytics/yolov5

### Setup

In [1]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker
from tqdm.notebook import tqdm

In [2]:
from sagemaker import get_execution_role
from sagemaker.session import Session

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket = 's3://descriptiveworld-models/CV_Models/'

In [3]:
print(region)
print(bucket)

us-west-2
s3://descriptiveworld-models/CV_Models/


In [4]:
# ensure that environment chosen for pytorch version >= 1.7 (as of 2021.11.01)

In [5]:
import torch
print(torch.__version__)

1.7.1


In [6]:
import sys
print(sys.version)

3.6.13 | packaged by conda-forge | (default, Feb 19 2021, 05:36:01) 
[GCC 9.3.0]


### Convert model to Torchscript

Run "aws configure" at command line to setup aws CLI.

In [8]:
!aws s3 cp s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt ../../yolov5/

download: s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt to ../../yolov5/best.pt


In [ ]:
%cd ../../yolov5
# not necessary to convert to torchscript
#!python ./bcj_export.py --weights ./best.pt

In [133]:
!cp best.pt model.pt

In [13]:
#!mkdir ./model
#!cp ./model.pt ./model/
#!mkdir ./model/code

In [14]:
!mkdir ./code

In [183]:
!cp ./utils/ -r ./code/utils/
!cp ./models/ -r ./code/models/
!cp ./data/ -r ./code/data/
!cp ./requirements.txt ./code/
!cp ./bcj_sm_ep_detect.py  ./code/

#### save model archive to s3

In [184]:
#!tar -czvf ./currentmodel.tar.gz ./current.torchscript.pt
!tar -czvf ./model.tar.gz ./model.pt
!aws s3 cp ./model.tar.gz s3://descriptiveworld-models/CV_Models/

./model.pt
upload: ./model.tar.gz to s3://descriptiveworld-models/CV_Models/model.tar.gz


### Create the model and endpoint

In [185]:
model_archive = './model.tar.gz'
prefix = 's3://descriptiveworld-models/CV_Models'
model_path = sess.upload_data(path=model_archive, key_prefix=prefix)
model_path

's3://sagemaker-us-west-2-769212126689/s3://descriptiveworld-models/CV_Models/model.tar.gz'

In [186]:
%%time
framework_version = '1.7.1'
py_version = 'py3'
instance_type = 'ml.t2.medium'
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

CPU times: user 22 µs, sys: 3 µs, total: 25 µs
Wall time: 30.3 µs


In [187]:
%%time
# reference:  https://stackoverflow.com/questions/68150444/aws-sagemaker-fails-loading-pytorch-pth-weights
# reference:  https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/model.py
# reference:  https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/model.py

sm_model = PyTorchModel(model_data=model_path,
                       framework_version=framework_version,
                       role=role,
                       sagemaker_session=sess,
                       entry_point='bcj_sm_ep_detect.py',
                       source_dir='code',
                       py_version=py_version,
                       env={"COMPILEDMODEL": 'False', 'MMS_MAX_RESPONSE_SIZE': '100000000', 'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'})
dw_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type)
print(sm_model.name)
print(dw_predictor.endpoint_name)

---------!pytorch-inference-2021-11-09-19-52-33-817
pytorch-inference-2021-11-09-19-52-34-097
CPU times: user 6.78 s, sys: 709 ms, total: 7.49 s
Wall time: 4min 40s


### Test Endpoint

In [188]:
client = boto3.client('sagemaker-runtime', region_name=region)
sample_img_url = "https://media.gq.com/photos/60f9c697101cc04fad71e5cf/master/pass/BEST-BASICS-1.jpg"
body = requests.get(sample_img_url).content
response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=body, ContentType=content_type)
pred_out = response['Body'].read().decode()

In [189]:
print(pred_out)

[
  {
    "class_num": 0,
    "class_name": "short sleeve top",
    "bbox": [
      0.1708303540945053,
      0.09795788675546646,
      0.08408801257610321,
      0.13775987923145294
    ],
    "conf": 0.25370076298713684
  },
  {
    "class_num": 3,
    "class_name": "shorts",
    "bbox": [
      0.04329350218176842,
      0.2529821991920471,
      0.07141232490539551,
      0.12472016364336014
    ],
    "conf": 0.2839449644088745
  },
  {
    "class_num": 1,
    "class_name": "trousers",
    "bbox": [
      0.1281137764453888,
      0.23411038517951965,
      0.045749299228191376,
      0.17297786474227905
    ],
    "conf": 0.30855464935302734
  },
  {
    "class_num": 2,
    "class_name": "long sleeve top",
    "bbox": [
      0.21237793564796448,
      0.2471497654914856,
      0.07431478798389435,
      0.14363721013069153
    ],
    "conf": 0.42144688963890076
  },
  {
    "class_num": 1,
    "class_name": "trousers",
    "bbox": [
      0.29174095392227173,
      0.2366363704

### Run Inference

In [100]:
%%time
iters = 1000
warmup = 100
client = boto3.client('sagemaker-runtime', region_name=region)

content_type = 'application/x-image'

sample_img_url = "https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg"
body = requests.get(sample_img_url).content

dw_perf = []
  
for i in tqdm(range(iters)):
    t0 = time.time()
    response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()
    #convert to millis
    dw_elapsed = (t1-t0)*1000
    
    if warmup == 0:
        dw_perf.append(uncompiled_elapsed)
    else:
        print(f'warmup ({i}, {iters}) : dw - {dw_elapsed} ms')
        warmup = warmup - 1

  0%|          | 0/1000 [00:00<?, ?it/s]

InternalFailure: An error occurred (InternalFailure) when calling the InvokeEndpoint operation (reached max retries: 4): An exception occurred while sending request to model. Please contact customer support regarding request ee0b7ab4-917f-40e6-b630-cf0e0d8f2c62.